In [15]:
import pandas as pd

# 1. kết quả bài thi
ANSWER_KEY = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D".split(',')

In [16]:
# 2. Xử lý file
def open_file():
    """
    Mở và đọc file dữ liệu
    Returns:
        tuple: (DataFrame, filename) hoặc (None, None) nếu có lỗi
    """
    while True:
        try:
            filename = input("Nhập tên lớp (ví dụ class1): ")
            if not filename.endswith('.txt'):
                filename += '.txt'
            filepath = f"Data Files/{filename}"  
            
            df = pd.read_csv(filepath, header=None)
            print(f"Successfully opened {filename}")
            return (df, filename)
                
        except FileNotFoundError:
            print("ko tìm thấy file.")
            retry = input("Bạn muốn làm lại không? (yes/no): ")
            if retry.lower() != 'yes':
                return (None, None)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            retry = input("Bạn muốn làm lại không ? (yes/no): ")
            if retry.lower() != 'yes':
                return (None, None)

def save_grades(filename, student_grades):
    """
    Lưu điểm của học sinh vào file *_grades (hàm này xử lý điểm của học sinh sau khi đã thống kê)
    """
    output_filename = filename.replace('.txt', '_grades.txt')
    output_path = f"Data Files/{output_filename}"
    
    try:
        with open(output_path, 'w') as f:
            for student_id, grade in student_grades.items():
                f.write(f"{student_id},{grade}\n")
        print(f"Tạo thành công {output_filename}")
    except Exception as e:
        print(f"Lỗi khi tạo file điểm: {str(e)}")

def read_grades_file(filename):
    """
    Đọc và hiển thị nội dung file *_grades theo format đề bài khi có yêu cầu
    """
    try:
        grades_filename = filename.replace('.txt', '_grades.txt')
        grades_filepath = f"Data Files/{grades_filename}"
        
        print(f"\n*** READING {grades_filename} ***")
        with open(grades_filepath, 'r') as f:
            lines = f.readlines()
            print("Student ID and Grades:")
            print("-" * 25)
            for line in lines:
                student_id, grade = line.strip().split(',')
                print(f"{student_id}: {grade}")
            print("-" * 25)
            print(f"Total students: {len(lines)}")
            
    except FileNotFoundError:
        print(f"Could not find grades file: {grades_filename}")
    except Exception as e:
        print(f"Error reading grades file: {str(e)}")

In [17]:
# 3. Kiểm tra dữ liệu
def validate_line(row, line_number):
    """
    Kiểm tra tính hợp lệ của một dòng dữ liệu
    """
    values = row.tolist()
    
    new_values = []
    for v in values:
        if str(v).lower() == 'nan':
            new_values.append('')
        else:
            new_values.append(str(v))
    values = new_values

    current_line = ','.join(values)

    if len(values) != 26:
        error_msg = f"Giá trị trong: Line {line_number} không đúng:\n{current_line}"
        return False, error_msg

    student_id = values[0]
    if (len(student_id) != 9 or 
        not student_id.startswith('N') or 
        not student_id[1:].isdigit()):
        error_msg = f"Giá trị ID trong: Line {line_number} không đúng:\n{current_line}"
        return False, error_msg
    
    return True, ""

In [18]:
# 4. Tính toán điểm thi
def calculate_score(student_answers):
    """
    Tính điểm cho một học sinh
    """
    score = 0
    for i, answer in enumerate(student_answers[1:]):
        if answer == '':  
            score += 0
        elif answer == ANSWER_KEY[i]:  
            score += 4
        else:  
            score -= 1
    return score

def calculate_median(scores):
    """
    Tính giá trị trung vị
    """
    sorted_scores = sorted(scores)
    n = len(sorted_scores)
    
    if n % 2 == 1:
        return sorted_scores[n//2]
    else:
        return (sorted_scores[n//2 - 1] + sorted_scores[n//2]) / 2

def calculate_statistics(scores):
    """
    Tính toán thống kê
    """
    mean = sum(scores) / len(scores)
    highest = max(scores)
    lowest = min(scores)
    score_range = highest - lowest
    median = calculate_median(scores)
    
    return mean, highest, lowest, score_range, median

In [19]:
# 5. Phân tích dữ liệu và ghi report
def analyze_data(df, filename):
    """
    Phân tích dữ liệu và tính điểm
    """
    print("**** ANALYZING ****")
    valid_count = 0
    invalid_count = 0
    scores = []
    student_grades = {}
    
    for i, row in df.iterrows():
        is_valid, error_message = validate_line(row, i+1)
        
        if is_valid:
            valid_count += 1
            student_answers = row.tolist()
            score = calculate_score(student_answers)
            scores.append(score)
            student_grades[student_answers[0]] = score
        else:
            invalid_count += 1
            print(error_message)
    
    if invalid_count == 0:
        print("No errors found!")
    
    print("**** REPORT ****")
    print(f"Total valid lines of data: {valid_count}")
    print(f"Total invalid lines of data: {invalid_count}")
    
    if scores:
        mean, highest, lowest, score_range, median = calculate_statistics(scores)
        print(f"Mean (average) score: {mean:.2f}")
        print(f"Highest score: {highest}")
        print(f"Lowest score: {lowest}")
        print(f"Range of scores: {score_range}")
        print(f"Median score: {int(median)}")
        
        save_grades(filename, student_grades)
    
    return valid_count, invalid_count

In [20]:
# 6. chạy chương trình
def main():
    df, filename = open_file()
    
    if df is not None and not df.empty:
        valid_count, invalid_count = analyze_data(df, filename)
        
        view_details = input("\nBạn muốn xem chi tiết không? (y/n): ")
        if view_details.lower() == 'y':
            read_grades_file(filename)
        else:
            print("Cám ơn")

if __name__ == "__main__":
    main()

Nhập tên lớp (ví dụ class1):  class1


Successfully opened class1.txt
**** ANALYZING ****
Giá trị ID trong: Line 1 không đúng:
N00x00001,A,A,D,D,C,D,D,A,,C,D,B,C,,B,C,B,D,A,C,,A,,C,D
**** REPORT ****
Total valid lines of data: 19
Total invalid lines of data: 1
Mean (average) score: 73.95
Highest score: 90
Lowest score: 65
Range of scores: 25
Median score: 70
Tạo thành công class1_grades.txt



Bạn muốn xem chi tiết không? (y/n):  y



*** READING class1_grades.txt ***
Student ID and Grades:
-------------------------
N00000002: 65
N00000003: 80
N00000004: 70
N00000005: 80
N00000006: 65
N00000007: 85
N00000008: 65
N00000009: 85
N00000010: 70
N00000011: 85
N00000012: 70
N00000013: 70
N00000014: 75
N00000015: 70
N00000016: 90
N00000017: 65
N00000018: 75
N00000019: 75
N00000020: 65
-------------------------
Total students: 19
